<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/BachChorales_HandsOnChapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
print(X)
print(y)

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
[40 50 60 70 80 90]


In [0]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
                                                                                   
print(yhat)

In [0]:
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
    return array(X), array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X)
print(y)
print(type(X))
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
[[[10 15 25]
  [20 25 45]
  [30 35 65]]]
[[40 45 85]]
<class 'numpy.ndarray'>
3


In [0]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu')) 
model.add(Dense(n_features)) 
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=1000, verbose=0)


In [0]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[197.93314 223.3317  407.88092]]


**Q10[part-1]. Download the Bach chorales dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. **bold text**

In [3]:
from google.colab import files
uploaded = files.upload()

Saving chorale_000.csv to chorale_000.csv


In [4]:
uploaded_validation = files.upload()

Saving chorale_229.csv to chorale_229.csv


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import numpy as np

X_train_000 = pd.read_csv('chorale_000.csv')
print(X_train_000.head(10))

X_valid_229 = pd.read_csv('chorale_229.csv')
print(X_valid_229.head(10))

X_train_000 = X_train_000.to_numpy()
X_valid_229 = X_valid_229.to_numpy()

dataX=[]
dataY=[]
dataValidationX=[]
dataValidationY=[]
for row in range(len(X_train_000)-1):
    dataX.append(X_train_000[row])
    dataY.append(X_train_000[row+1])
print(dataX[0:5])
print(dataY[0:5])

for row in range(len(X_valid_229)-1):
     dataValidationX.append(X_valid_229[row])
     dataValidationY.append(X_valid_229[row+1])
print(dataValidationX[0:5])
print(dataValidationY[0:5])

   note0  note1  note2  note3
0     74     70     65     58
1     74     70     65     58
2     74     70     65     58
3     74     70     65     58
4     75     70     58     55
5     75     70     58     55
6     75     70     60     55
7     75     70     60     55
8     77     69     62     50
9     77     69     62     50
   note0  note1  note2  note3
0     72     67     60     48
1     72     67     60     48
2     72     67     60     48
3     72     67     60     48
4     72     67     64     48
5     72     67     64     48
6     72     67     64     50
7     72     67     64     50
8     72     67     64     52
9     72     67     64     52
[array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([75, 70, 58, 55])]
[array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([74, 70, 65, 58]), array([75, 70, 58, 55]), array([75, 70, 58, 55])]
[array([72, 67, 60, 48]), array([72, 67, 60, 48]), array([72, 67, 60, 48]), array([72, 6

In [8]:
"""X = np.reshape(dataX, (len(dataX), 1, 4))
X_train, X_test, y_train,  y_test = train_test_split(X, dataY, test_size=0.33, random_state=42)
print(X_train[0:5])
print(y_train[0:5])
"""
# convert into input/output
i,j=0,0
k,l=0,0
X, y=[], []
validX, validY=[], []

while i<len(dataX):
  X.append(np.array([dataX[i]]))
  i += 1

while j<len(dataY):
  y.append(dataY[j])
  j+= 1

while k<len(dataValidationX):
  validX.append(np.array([dataValidationX[k]]))
  k += 1

while l<len(dataValidationY):
  validY.append(dataValidationY[l])
  l+=1

X=np.asarray(X)
y=np.asarray(y)
print(X[0:2])
print(y[0])
print(X.shape, y.shape)


validX=np.asarray(validX)
validY=np.asarray(validY)
print(validX[0:2])
print(validY[0])
print(validX.shape, validY.shape)

[[[74 70 65 58]]

 [[74 70 65 58]]]
[74 70 65 58]
(191, 1, 4) (191, 4)
[[[72 67 60 48]]

 [[72 67 60 48]]]
[72 67 60 48]
(195, 1, 4) (195, 4)


In [39]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

# choose a number of time steps
n_steps = None

model_lstm = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences=True, input_shape=[n_steps, n_features]),
    keras.layers.LSTM(100),
    keras.layers.Dense(n_features)
])
model_lstm.compile(optimizer='adam', loss='mse')
# fit model
model_lstm.fit(X, y, epochs=250, validation_data=(validX, validY))

4
Train on 191 samples, validate on 195 samples
Epoch 1/250
191/191 [==============================] - 6s 31ms/sample - loss: 4042.0204 - val_loss: 4059.1162
Epoch 2/250
191/191 [==============================] - 0s 338us/sample - loss: 4004.8116 - val_loss: 4016.6945
Epoch 3/250
191/191 [==============================] - 0s 308us/sample - loss: 3958.0647 - val_loss: 3963.0906
Epoch 4/250
191/191 [==============================] - 0s 281us/sample - loss: 3899.7893 - val_loss: 3897.5774
Epoch 5/250
191/191 [==============================] - 0s 310us/sample - loss: 3829.8375 - val_loss: 3820.2122
Epoch 6/250
191/191 [==============================] - 0s 286us/sample - loss: 3747.4558 - val_loss: 3729.8120
Epoch 7/250
191/191 [==============================] - 0s 329us/sample - loss: 3652.1763 - val_loss: 3627.2746
Epoch 8/250
191/191 [==============================] - 0s 329us/sample - loss: 3546.2367 - val_loss: 3514.7981
Epoch 9/250
191/191 [==============================] - 0s 345us/s

"\n# define model\nmodel_lstm = Sequential()\nmodel_lstm.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))\nmodel_lstm.add(LSTM(100, activation='relu')) \nmodel_lstm.add(Dense(n_features))\nmodel_lstm.compile(optimizer='adam', loss='mse')\n# fit model\nmodel_lstm.fit(X, y, epochs=100, validation_data=(validX, validY))\n"

In [41]:
# demonstrate prediction
x_input = X[0:5]
x_input = x_input.reshape((1, len(x_input), n_features))
print(x_input)
yhat = model_lstm.predict(x_input, verbose=0)
print(yhat)

[[[74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [75 70 58 55]]]
[[92.83752 85.1502  76.7752  66.26208]]


**Q10[part-2] Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out Google’s Coconet model, which was used for a nice Google doodle about Bach.**

In [42]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = X[0]
x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  #x_input = x_input.reshape((1, len(x_input), n_features))
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_lstm.predict(x_input, verbose=1)
  print('Predicated Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  #print(x_input, yhat)
  #print(x_input.shape, yhat.shape)
  x_input = np.concatenate([x_input, yhat], axis=1)
  #print(x_input.shape)
  #print(x_input)
  i += 1
  if i>10:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[71.821106 65.907    59.41087  51.26083 ]]



Input:  [[[74.         70.         65.         58.        ]
  [71.82110596 65.90699768 59.4108696  51.26082993]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[87.93178 80.65569 72.70713 62.76358]]



Input:  [[[74.         70.         65.         58.        ]
  [71.82110596 65.90699768 59.4108696  51.26082993]
  [87.93177795 80.65569305 72.70713043 62.76358032]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[91.43956 83.86755 75.6096  65.26723]]



Input:  [[[74.         70.         65.         58.        ]
  [71.82110596 65.90699768 59.4108696  51.26082993]
  [87.93177795 80.65569305 72.70713043

In [19]:
# Using SimpleRNN 
print(n_features)
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, input_shape=[None, n_features]),
    keras.layers.SimpleRNN(100),
    keras.layers.Dense(n_features)
])

model_rnn.compile(loss="mse", optimizer="adam")
print(X.shape, y.shape)
history = model_rnn.fit(X, y, epochs=500,
                    validation_data=(validX, validY))

4
(191, 1, 4) (191, 4)
Train on 191 samples, validate on 195 samples
Epoch 1/500
191/191 [==============================] - 1s 7ms/sample - loss: 3935.9916 - val_loss: 3808.4798
Epoch 2/500
191/191 [==============================] - 0s 194us/sample - loss: 3676.4835 - val_loss: 3588.0970
Epoch 3/500
191/191 [==============================] - 0s 152us/sample - loss: 3483.4858 - val_loss: 3431.0806
Epoch 4/500
191/191 [==============================] - 0s 148us/sample - loss: 3345.2145 - val_loss: 3312.7697
Epoch 5/500
191/191 [==============================] - 0s 146us/sample - loss: 3237.2155 - val_loss: 3215.7495
Epoch 6/500
191/191 [==============================] - 0s 148us/sample - loss: 3146.8364 - val_loss: 3132.8675
Epoch 7/500
191/191 [==============================] - 0s 147us/sample - loss: 3067.4515 - val_loss: 3056.0294
Epoch 8/500
191/191 [==============================] - 0s 152us/sample - loss: 2992.5083 - val_loss: 2982.3813
Epoch 9/500
191/191 [========================

In [20]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = X[0]
x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  #x_input = x_input.reshape((1, len(x_input), n_features))
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_rnn.predict(x_input, verbose=1)
  print('Predicted Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 386ms/sample
Predicted Output:  [[74.03013  67.03655  59.808205 51.330708]]



Input:  [[[74.         70.         65.         58.        ]
  [74.03012848 67.03655243 59.80820465 51.33070755]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.00299  67.00816  59.785255 51.309227]]



Input:  [[[74.         70.         65.         58.        ]
  [74.03012848 67.03655243 59.80820465 51.33070755]
  [74.00299072 67.00816345 59.78525543 51.30922699]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.00301  67.008194 59.785267 51.30924 ]]



Input:  [[[74.         70.         65.         58.        ]
  [74.03012848 67.03655243 59.80820465 51.33070755]
  [74.00299072 67.00816345 59.7